In [1]:
import lxw
import json

# 1. Tool Call Sequence

In [56]:
def List_Files(directory):
    if directory == 'c:/':
        return ['范利波.txt', '吕小望.txt', '安鑫.avi' ]
    return []

In [57]:
def Get_File_Content(file_path):
    if '范利波' in file_path:
        return '范立波110031201301010250'
    if '吕小望' in file_path:
        return '吕小望13578781231'
    return ''
    

In [12]:
tools = [
    {
        'type': 'function',
        'function': {
            'name': 'List_Files',
            'description': 'List all files in the specified directory.',
            'parameters': {
                'type': 'object',
                'properties': {
                    'directory': {
                        'type': 'string',
                        'description': 'the directory path, e.g. c:/abc/d'
                    }
                },
                'required': ['directory']
            }
        }
    },
    {
        'type': 'function',
        'function': {
            'name': 'Get_File_Content',
            'description': 'Get the text content of specified file_path.',
            'parameters': {
                'type': 'object',
                'properties': {
                    'file_path': {
                        'type': 'string',
                        'description': 'the full file path, e.g. c:/abc/d.txt'
                    }
                },
                'required': ['file_path']
            }
        }
    },    
]

In [60]:
messages = [
    {
        'role': 'system',
        'content': 'Do not make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.',
    },
    {
        "role": "user",
        "content": '列出目录 c:/ 下，所有的文件',
    }
]

In [61]:
model = lxw.gpt4_turbo_0125
client = lxw.new_client(model = model)

In [62]:
response1 = client.chat.completions.create(
    model = model,
    messages = messages,
    tools = tools
)

In [63]:
print(response1.choices[0])

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_uclh8MeHPqpMMYZhdUQcoOo1', function=Function(arguments='{"directory":"c:/"}', name='List_Files'), type='function')]), content_filter_results={})


In [64]:
message = response1.choices[0].message
messages.append(message)
files = '\r\n'.join(List_Files('c:/'))
messages.append({
    'role': 'tool',
    'tool_call_id': message.tool_calls[0].id,
    'name': message.tool_calls[0].function.name,
    'content': files
})
response2 = client.chat.completions.create(
    model = model,
    messages = messages,
    tools = tools
)


In [65]:
messages

[{'role': 'system',
  'content': 'Do not make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.'},
 {'role': 'user', 'content': '列出目录 c:/ 下，所有的文件'},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_uclh8MeHPqpMMYZhdUQcoOo1', function=Function(arguments='{"directory":"c:/"}', name='List_Files'), type='function')]),
 {'role': 'tool',
  'tool_call_id': 'call_uclh8MeHPqpMMYZhdUQcoOo1',
  'name': 'List_Files',
  'content': '范利波.txt\r\n吕小望.txt\r\n安鑫.avi'}]

In [67]:
print(response2.choices[0].message.content)

在目录 `c:/` 下，找到的所有文件如下：

1. 范利波.txt
2. 吕小望.txt
3. 安鑫.avi


In [76]:
type(response1.choices[0].message.tool_calls[0].function.arguments)

str

# 2. Tool Call

In [86]:
def chat_completion_with_tools_call(prompt):
    messages = [
        {
            'role': 'system',
            'content': 'Do not make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.',
        },
        {
            "role": "user",
            "content": prompt,
        }
    ]
    
    model = lxw.gpt4_turbo_0125
    client = lxw.new_client(model = model)

    response = client.chat.completions.create(
        model = model,
        messages = messages,
        tools = tools
    )
    
    while response.choices[0].finish_reason == 'tool_calls':
        messages.append(response.choices[0].message)
        for tool_call in response.choices[0].message.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            print(function_name, function_args)
            if function_name == 'List_Files':
                directory = function_args.get('directory')
                function_result = '\r\n'.join(List_Files(directory))
            elif function_name == 'Get_File_Content':
                file_path = function_args.get('file_path')
                function_result = Get_File_Content(file_path)
            else:
                raise ValueError('invalid function ' + function_name)
            messages.append({
                'role': 'tool',
                'tool_call_id': tool_call.id,
                'name': function_name,
                'content': function_result,
            })
        response = client.chat.completions.create(
            model = model,
            messages = messages,
            tools = tools
        )
    return response.choices[0].message.content

In [83]:
print(chat_completion_with_tools_call('列举 c:/ 下所有文件'))

List_Files {'directory': 'c:/'}
在 `c:/` 目录下，存在以下文件：

1. 范利波.txt
2. 吕小望.txt
3. 安鑫.avi


In [84]:
print(chat_completion_with_tools_call('列举 c:/ 下所有视频文件'))

List_Files {'directory': 'c:/'}
在目录 `c:/` 下找到的视频文件为：
- 安鑫.avi


In [89]:
print(chat_completion_with_tools_call('搜索 c:/ 下所有内容包含18位身份证号的文件'))

List_Files {'directory': 'c:/'}
Get_File_Content {'file_path': 'c:/范利波.txt'}
Get_File_Content {'file_path': 'c:/吕小望.txt'}
Get_File_Content {'file_path': 'c:/安鑫.avi'}
在 `c:/` 路径下，包含18位身份证号的文件为:

- `范利波.txt` 其内容包含18位身份证号 `110031201301010250`


# 3. 万能工具

In [2]:
python_script_tool = {
    'type': 'function',
    'function': {
        'name': 'Execute_Python_Script',
        'description': 'Run a Python script and return the output of the execution. In the script, use Python print() function to produce the output.',
        'parameters': {
            'type': 'object',
            'properties': {
                'script': {
                    'type': 'string',
                    'description': 'The Python script to be executed.'
                }
            },
            'required': ['script']
        }
    }
}

In [3]:
def Execute_Python_Script(script):
    import sys
    import io
    print_hook = io.StringIO()
    print_save = sys.stdout
    sys.stdout = print_hook
    try:
        local_env = {}
        exec(script, local_env)
    finally:
        sys.stdout = print_save
    print_hook.seek(0)
    return print_hook.read()


In [4]:
def chat_completion_with_python_call(prompt):
    messages = [
        {
            'role': 'system',
            'content': 'Where possible, try to use the Execute_Python_Script tool to provide an answer. You can use Python [requests] package to fetch content from the internet.',
        },
        {
            "role": "user",
            "content": prompt,
        }
    ]
    
    model = lxw.gpt4_turbo_0125
    client = lxw.new_client(model = model)

    response = client.chat.completions.create(
        model = model,
        messages = messages,
        tools = [python_script_tool]
    )
    
    while response.choices[0].finish_reason == 'tool_calls':
        messages.append(response.choices[0].message)
        for tool_call in response.choices[0].message.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            print(function_name)
            if function_name == 'Execute_Python_Script':
                script = function_args.get('script')
                print('script: \r\n', script)
                function_result = Execute_Python_Script(script)
            else:
                raise ValueError('invalid function ' + function_name)
            print('result:\r\n', function_result)
            messages.append({
                'role': 'tool',
                'tool_call_id': tool_call.id,
                'name': function_name,
                'content': function_result,
            })
        response = client.chat.completions.create(
            model = model,
            messages = messages,
            tools = tools
        )
    return response.choices[0].message.content

In [128]:
print(chat_completion_with_python_call('将 12ABC 转为小写'))

Execute_Python_Script
script: 
 print('12ABC'.lower())
result:
 12abc

将 "12ABC" 转换为小写后结果为 "12abc"。


In [129]:
print(chat_completion_with_python_call('1000以内，最大的质数是多少'))

Execute_Python_Script
script: 
 def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True

max_prime = 2
for i in range(3, 1001, 2):
    if is_prime(i):
        max_prime = i

print(max_prime)
result:
 997

1000以内最大的质数是997。


In [130]:
print(chat_completion_with_python_call('今天是星期几'))

Execute_Python_Script
script: 
 from datetime import datetime

# Get today's date
today = datetime.now()

# Get the day of the week
weekday = today.strftime('%A')

print(weekday)
result:
 Tuesday

今天是星期二。


In [131]:
print(chat_completion_with_python_call('字符串\'abcd\'的sha1哈希值是多少'))

Execute_Python_Script
script: 
 import hashlib

# 文字串 'abcd'
string = 'abcd'

# 计算 SHA1 哈希值
hash_object = hashlib.sha1(string.encode())
hash_hex = hash_object.hexdigest()

print(hash_hex)
result:
 81fe8bfe87576c3ecb22426f8e57847382917acf

字符串 'abcd' 的 SHA1 哈希值是 `81fe8bfe87576c3ecb22426f8e57847382917acf`。


In [137]:
print(chat_completion_with_python_call('用百度搜索一下有关龙湖的内容，总结一下最主要的结果'))

Execute_Python_Script
script: 
 import requests
from bs4 import BeautifulSoup

# Define the header to mimic a browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

# Define the search URL for Baidu
search_url = 'https://www.baidu.com/s?wd=龙湖'

# Sending the GET request
response = requests.get(search_url, headers=headers)

# Parsing the response content
soup = BeautifulSoup(response.text, 'html.parser')

# Extracting and printing the main content
for item in soup.find_all('div', class_='c-container')[:5]: # Limiting to the first 5 results
    title = item.find('h3').get_text()
    summary = item.find('div', class_='c-abstract').get_text() if item.find('div', class_='c-abstract') else 'No summary available'
    print(f'Title: {title}\nSummary: {summary}\n')

result:
 Title: 北京丰台区龙湖建工九里熙宸售楼处-2024房价走势
Summary: 龙湖龙湖建工九里熙宸售楼处,打造品质生活!洋房、别墅两相宜,价格实惠.购房新选择,龙湖龙湖建工九里熙宸售楼处,专业房产中介为您服务!

Tit

In [140]:
print(chat_completion_with_python_call('目前港币的汇率是多少？'))

Execute_Python_Script
script: 
 import requests

# 汇率查询的API URL
url = 'https://api.exchangerate-api.com/v4/latest/HKD'

# 发送GET请求
response = requests.get(url)
# 将响应的内容解析成JSON
rates = response.json().get('rates', {})

# 打印美元和人民币的汇率
print(f"港币兑美元汇率是: {rates.get('USD', '无法获取')}")
print(f"港币兑人民币汇率是: {rates.get('CNY', '无法获取')}")
result:
 港币兑美元汇率是: 0.128
港币兑人民币汇率是: 0.927

目前港币的汇率如下：
- 港币兑美元汇率是：0.128
- 港币兑人民币汇率是：0.927


In [7]:
print(chat_completion_with_python_call('今天北京的天气怎么样？'))

Execute_Python_Script
script: 
 import requests

# Define the API URL
url = 'http://api.openweathermap.org/data/2.5/weather?q=Beijing&appid=YOUR_API_KEY&units=metric'

# Send GET request to the API
response = requests.get(url)

# Parse the JSON response
weather_data = response.json()

# Extract relevant information
temperature = weather_data['main']['temp']
description = weather_data['weather'][0]['description']

# Print the weather details
print(f'The current temperature in Beijing is {temperature}°C and the weather is described as {description}.')


KeyError: 'main'

In [143]:
print(chat_completion_with_python_call('基金 华夏成长混合 的当前净值是多少？'))

Execute_Python_Script
script: 
 import requests
response = requests.get('http://fund.eastmoney.com/js/fundcode_search.js')
all_funds = eval(response.text.strip().replace('var r = ', '').replace(';', ''))
for fund in all_funds:
    if fund[2] == '华夏成长混合':
        fund_code = fund[0]
        break

fund_info_url = f'http://fundgz.1234567.com.cn/js/{fund_code}.js?rt=1463558676006'
response = requests.get(fund_info_url)
json_data = eval(response.text.strip().replace('jsonpgz(', '').replace(');', ''))
print(json_data['gsz'])
result:
 0.7696

华夏成长混合基金的当前净值大约是 0.7696。
